In [1]:
# Copyright (c) OpenMMLab. All rights reserved.
from argparse import ArgumentParser
from typing import Dict
import sys

sys.path.append('/home/moriki/PoseEstimation/mmpose/')

from mmpose.apis.inferencers import MMPoseInferencer, get_model_aliases

filter_args = dict(bbox_thr=0.3, nms_thr=0.3, pose_based_nms=False)
POSE2D_SPECIFIC_ARGS = dict(
    yoloxpose=dict(bbox_thr=0.01, nms_thr=0.65, pose_based_nms=True),
    rtmo=dict(bbox_thr=0.1, nms_thr=0.65, pose_based_nms=True),
)


def parse_args():
    parser = ArgumentParser()
    parser.add_argument(
        'inputs',
        type=str,
        nargs='?',
        help='Input image/video path or folder path.')

    # init args
    parser.add_argument(
        '--pose2d',
        type=str,
        default=None,
        help='Pretrained 2D pose estimation algorithm. It\'s the path to the '
        'config file or the model name defined in metafile.')
    parser.add_argument(
        '--pose2d-weights',
        type=str,
        default=None,
        help='Path to the custom checkpoint file of the selected pose model. '
        'If it is not specified and "pose2d" is a model name of metafile, '
        'the weights will be loaded from metafile.')
    parser.add_argument(
        '--pose3d',
        type=str,
        default=None,
        help='Pretrained 3D pose estimation algorithm. It\'s the path to the '
        'config file or the model name defined in metafile.')
    parser.add_argument(
        '--pose3d-weights',
        type=str,
        default=None,
        help='Path to the custom checkpoint file of the selected pose model. '
        'If it is not specified and "pose3d" is a model name of metafile, '
        'the weights will be loaded from metafile.')
    parser.add_argument(
        '--det-model',
        type=str,
        default=None,
        help='Config path or alias of detection model.')
    parser.add_argument(
        '--det-weights',
        type=str,
        default=None,
        help='Path to the checkpoints of detection model.')
    parser.add_argument(
        '--det-cat-ids',
        type=int,
        nargs='+',
        default=0,
        help='Category id for detection model.')
    parser.add_argument(
        '--scope',
        type=str,
        default='mmpose',
        help='Scope where modules are defined.')
    parser.add_argument(
        '--device',
        type=str,
        default=None,
        help='Device used for inference. '
        'If not specified, the available device will be automatically used.')
    parser.add_argument(
        '--show-progress',
        action='store_true',
        help='Display the progress bar during inference.')

    # The default arguments for prediction filtering differ for top-down
    # and bottom-up models. We assign the default arguments according to the
    # selected pose2d model
    args, _ = parser.parse_known_args()
    for model in POSE2D_SPECIFIC_ARGS:
        if model in args.pose2d:
            filter_args.update(POSE2D_SPECIFIC_ARGS[model])
            break

    # call args
    parser.add_argument(
        '--show',
        action='store_true',
        help='Display the image/video in a popup window.')
    parser.add_argument(
        '--draw-bbox',
        action='store_true',
        help='Whether to draw the bounding boxes.')
    parser.add_argument(
        '--draw-heatmap',
        action='store_true',
        default=False,
        help='Whether to draw the predicted heatmaps.')
    parser.add_argument(
        '--bbox-thr',
        type=float,
        default=filter_args['bbox_thr'],
        help='Bounding box score threshold')
    parser.add_argument(
        '--nms-thr',
        type=float,
        default=filter_args['nms_thr'],
        help='IoU threshold for bounding box NMS')
    parser.add_argument(
        '--pose-based-nms',
        type=lambda arg: arg.lower() in ('true', 'yes', 't', 'y', '1'),
        default=filter_args['pose_based_nms'],
        help='Whether to use pose-based NMS')
    parser.add_argument(
        '--kpt-thr', type=float, default=0.3, help='Keypoint score threshold')
    parser.add_argument(
        '--tracking-thr', type=float, default=0.3, help='Tracking threshold')
    parser.add_argument(
        '--use-oks-tracking',
        action='store_true',
        help='Whether to use OKS as similarity in tracking')
    parser.add_argument(
        '--disable-norm-pose-2d',
        action='store_true',
        help='Whether to scale the bbox (along with the 2D pose) to the '
        'average bbox scale of the dataset, and move the bbox (along with the '
        '2D pose) to the average bbox center of the dataset. This is useful '
        'when bbox is small, especially in multi-person scenarios.')
    parser.add_argument(
        '--disable-rebase-keypoint',
        action='store_true',
        default=False,
        help='Whether to disable rebasing the predicted 3D pose so its '
        'lowest keypoint has a height of 0 (landing on the ground). Rebase '
        'is useful for visualization when the model do not predict the '
        'global position of the 3D pose.')
    parser.add_argument(
        '--num-instances',
        type=int,
        default=1,
        help='The number of 3D poses to be visualized in every frame. If '
        'less than 0, it will be set to the number of pose results in the '
        'first frame.')
    parser.add_argument(
        '--radius',
        type=int,
        default=3,
        help='Keypoint radius for visualization.')
    parser.add_argument(
        '--thickness',
        type=int,
        default=1,
        help='Link thickness for visualization.')
    parser.add_argument(
        '--skeleton-style',
        default='mmpose',
        type=str,
        choices=['mmpose', 'openpose'],
        help='Skeleton style selection')
    parser.add_argument(
        '--black-background',
        action='store_true',
        help='Plot predictions on a black image')
    parser.add_argument(
        '--vis-out-dir',
        type=str,
        default='',
        help='Directory for saving visualized results.')
    parser.add_argument(
        '--pred-out-dir',
        type=str,
        default='',
        help='Directory for saving inference results.')
    parser.add_argument(
        '--show-alias',
        action='store_true',
        help='Display all the available model aliases.')

    call_args = vars(parser.parse_args())

    init_kws = [
        'pose2d', 'pose2d_weights', 'scope', 'device', 'det_model',
        'det_weights', 'det_cat_ids', 'pose3d', 'pose3d_weights',
        'show_progress'
    ]
    init_args = {}
    for init_kw in init_kws:
        init_args[init_kw] = call_args.pop(init_kw)

    display_alias = call_args.pop('show_alias')

    return init_args, call_args, display_alias


def display_model_aliases(model_aliases: Dict[str, str]) -> None:
    """
    Display the available model aliases and their corresponding model names.
    """
    aliases = list(model_aliases.keys())
    max_alias_length = max(map(len, aliases))
    print(f'{"ALIAS".ljust(max_alias_length+2)}MODEL_NAME')
    for alias in sorted(aliases):
        print(f'{alias.ljust(max_alias_length+2)}{model_aliases[alias]}')


def main():
    init_args, call_args, display_alias = parse_args()
    if display_alias:
        model_alises = get_model_aliases(init_args['scope'])
        display_model_aliases(model_alises)
    else:
        inferencer = MMPoseInferencer(**init_args)
        for _ in inferencer(**call_args):
            pass


if __name__ == '__main__':
    main()


/home/moriki/.pyenv/versions/3.9.7/envs/mmpose-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: argument of type 'NoneType' is not iterable

In [2]:
# 必要なライブラリのインポート
import sys
sys.path.append('/home/moriki/PoseEstimation/mmpose/')

from mmpose.apis.inferencers import MMPoseInferencer, get_model_aliases

# コマンドライン引数を直接設定
def run_pose_estimation(
    input_image,
    pose2d='rtmo',
    pose2d_weights=None,
    det_model=None,
    det_weights=None,
    det_cat_ids=0,
    device=None,
    show_progress=False,
    show=False,
    draw_bbox=False,
    draw_heatmap=False,
    vis_out_dir='',
    pred_out_dir='',
    scope='mmpose'
):
    init_args = {
        'pose2d': pose2d,
        'pose2d_weights': pose2d_weights,
        'device': device,
        'scope': scope,
        'det_model': det_model,
        'det_weights': det_weights,
        'det_cat_ids': det_cat_ids,
        'show_progress': show_progress
    }
    
    call_args = {
        'show': show,
        'draw_bbox': draw_bbox,
        'draw_heatmap': draw_heatmap,
        'vis_out_dir': vis_out_dir,
        'pred_out_dir': pred_out_dir,
        'inputs': input_image
    }

    if show:
        model_alises = get_model_aliases(scope)
        display_model_aliases(model_alises)
    else:
        inferencer = MMPoseInferencer(**init_args)
        for result in inferencer(**call_args):
            pass  # 結果を処理または表示

# 使用例
run_pose_estimation(
    input_image='/home/moriki/PoseEstimation/mmpose/data/pose/CrowdPose/images-origin/100000.jpg',
    vis_out_dir='vis_results',
    show=True
)


ALIAS      MODEL_NAME
animal     rtmpose-m_8xb64-210e_ap10k-256x256
body       rtmpose-m_8xb256-420e_body8-256x192
body17     rtmpose-m_8xb256-420e_body8-256x192
body26     rtmpose-m_8xb512-700e_body8-halpe26-256x192
edpose     edpose_res50_8xb2-50e_coco-800x1333
face       rtmpose-m_8xb64-120e_lapa-256x256
hand       rtmpose-m_8xb256-210e_hand5-256x256
hand3d     internet_res50_4xb16-20e_interhand3d-256x256
human      rtmpose-m_8xb256-420e_body8-256x192
human3d    motionbert_dstformer-ft-243frm_8xb32-120e_h36m
rtmo       rtmo-l_16xb16-600e_body7-640x640
rtmpose-l  rtmpose-l_8xb256-420e_body8-384x288
vitpose    td-hm_ViTPose-base-simple_8xb64-210e_coco-256x192
vitpose-b  td-hm_ViTPose-base-simple_8xb64-210e_coco-256x192
vitpose-h  td-hm_ViTPose-huge-simple_8xb64-210e_coco-256x192
vitpose-l  td-hm_ViTPose-large-simple_8xb64-210e_coco-256x192
vitpose-s  td-hm_ViTPose-small-simple_8xb64-210e_coco-256x192
wholebody  rtmw-m_8xb1024-270e_cocktail14-256x192


In [3]:
run_pose_estimation(
    input_image='/home/moriki/PoseEstimation/mmpose/data/pose/CrowdPose/images-origin/100000.jpg',
    vis_out_dir='vis_results',
    show=False
)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmo/rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.pth


FileNotFoundError: The metainfo config file "configs/_base_/datasets/coco.py" does not exist.

In [5]:
def main(input_image, pose2d=None, pose2d_weights=None, pose3d=None, pose3d_weights=None,
         det_model=None, det_weights=None, det_cat_ids=0, scope='mmpose', device=None,
         show_progress=False, show=False, draw_bbox=False, draw_heatmap=False,
         vis_out_dir='', pred_out_dir='', display_alias=False):
    init_args = {
        'pose2d': pose2d,
        'pose2d_weights': pose2d_weights,
        'pose3d': pose3d,
        'pose3d_weights': pose3d_weights,
        'device': device,
        'scope': scope,
        'det_model': det_model,
        'det_weights': det_weights,
        'det_cat_ids': det_cat_ids,
        'show_progress': show_progress
    }

    call_args = {
        'show': show,
        'draw_bbox': draw_bbox,
        'draw_heatmap': draw_heatmap,
        'vis_out_dir': vis_out_dir,
        'pred_out_dir': pred_out_dir,
        'inputs': input_image
    }

    if display_alias:
        model_aliases = get_model_aliases(scope)
        display_model_aliases(model_aliases)
    else:
        inferencer = MMPoseInferencer(**init_args)
        for _ in inferencer(**call_args):
            pass

# Notebook内での実行例
main(
    input_image='/home/moriki/PoseEstimation/mmpose/data/pose/CrowdPose/images-origin/100000.jpg',
    pose2d='rtmo',
    pose2d_weights='/home/moriki/PoseEstimation/mmpose/models/rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.pth',
    vis_out_dir='vis_results',
    show=True
)


Loads checkpoint by local backend from path: /home/moriki/PoseEstimation/mmpose/models/rtmo-l_16xb16-600e_body7-640x640-b37118ce_20231211.pth


FileNotFoundError: The metainfo config file "configs/_base_/datasets/coco.py" does not exist.